In [2]:
import pandas as pd
import numpy as np

In [3]:
data_path = "C:/Users/armcl/Python/Springboard_Cap1/Data/" #Home
#data_path = "C:/Users/yyzhao/python/Springboard/Capstone1/Data/" #Work

In [32]:
#Read Redfin data
redfin = pd.read_csv(data_path + "Redfin/Santa_Clara_County.csv")
#Remove irrelvant and poor quality (for example 'DAYS ON MARKET') data columns
redfin = redfin.drop(columns = ['SALE TYPE', 'DAYS ON MARKET', 'STATUS', 'NEXT OPEN HOUSE START TIME', 
                       'NEXT OPEN HOUSE END TIME', 'SOURCE', 'FAVORITE', 'INTERESTED', 
                       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
                               'LOCATION', 'MLS#', '$/SQUARE FEET'])

redfin.head(2)

,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
0,July-2-2019,Single Family Residential,788 Orkney Ave,SANTA CLARA,CA,95054,1050000.0,3.0,2.0,1097.0,6242.0,1961.0,957.0,NaN,37.389165,-121.947778
1,April-19-2018,Condo/Co-op,3901 Lick Mill Blvd #259,SANTA CLARA,CA,95054,882000.0,1.0,1.0,865.0,865.0,2005.0,1020.0,383.0,37.396173,-121.943140


In [33]:
#Seems only Property Type, State, Latitude and Longitude have non NAs
redfin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29919 entries, 0 to 29918
Data columns (total 16 columns):
SOLD DATE             25996 non-null object
PROPERTY TYPE         29919 non-null object
ADDRESS               29800 non-null object
CITY                  29899 non-null object
STATE OR PROVINCE     29919 non-null object
ZIP OR POSTAL CODE    29902 non-null object
PRICE                 29789 non-null float64
BEDS                  29465 non-null float64
BATHS                 28878 non-null float64
SQUARE FEET           29768 non-null float64
LOT SIZE              28491 non-null float64
YEAR BUILT            29747 non-null float64
$/SQUARE FEET         29640 non-null float64
HOA/MONTH             9695 non-null float64
LATITUDE              29919 non-null float64
LONGITUDE             29919 non-null float64
dtypes: float64(10), object(6)
memory usage: 3.7+ MB


In [34]:
redfin.describe()
#Observations of data issues
#Min and Max suggests outliers, need further investigations
#Check out different lower percentiles and drop from there

#There are duplicate data while downloading when setting filters, need to be removed

#For type multifamily (5+ unit) but beds < 5 or missing, should be removed
#Should we only focus on type SFH, Townhouse, and Condo?

#There are observations with state in PA or TX, should be removed

#Sorted by price, some observations with super high prices are apartments/office/land and should be removed
#Observations with prices missing or too low are bad data, need to be removed

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
count,2.978900e+04,29465.000000,28878.000000,29768.000000,2.849100e+04,29747.000000,2.964000e+04,9695.000000,29919.000000,29919.000000
mean,1.452307e+06,3.290582,2.341246,1888.215298,1.013576e+04,1974.448516,8.681001e+02,324.167406,37.299072,-121.898079
std,2.739201e+06,1.090651,0.846894,3846.153051,4.236792e+04,23.265612,1.063604e+04,203.997016,0.093041,0.127440
min,3.895000e+03,0.000000,0.500000,0.000000,1.000000e+00,1858.000000,3.000000e+00,1.000000,36.964850,-122.180281
25%,8.500000e+05,3.000000,2.000000,1234.000000,3.060000e+03,1959.000000,5.630000e+02,208.000000,37.257107,-121.982254
50%,1.155000e+06,3.000000,2.000000,1593.500000,6.103000e+03,1972.000000,7.195000e+02,325.000000,37.310587,-121.900109
75%,1.600000e+06,4.000000,3.000000,2106.000000,8.001500e+03,1990.000000,9.420000e+02,412.000000,37.362173,-121.834079
max,2.555000e+08,12.000000,10.000000,350451.000000,2.652368e+06,2019.000000,1.730000e+06,6000.000000,37.465350,-121.419033


In [35]:
#The pricer max outliers are mostly Multi-Family (5+ Unit) property type
#Check the size, price grouped by property type
redfin.groupby('PROPERTY TYPE').size()

PROPERTY TYPE
Condo/Co-op                   5232
Multi-Family (2-4 Unit)        784
Multi-Family (5+ Unit)         160
Other                            3
Single Family Residential    20467
Townhouse                     3264
Vacant Land                      9
dtype: int64

In [36]:
redfin.groupby('PROPERTY TYPE').mean()

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
PROPERTY TYPE,,,,,,,,,,
Condo/Co-op,8.243012e+05,2.106113,1.914376,1180.511825,2379.989378,1989.692055,822.537481,418.182382,37.325797,-121.912794
Multi-Family (2-4 Unit),1.509862e+06,5.631673,3.292553,2509.993606,9278.582908,1957.822023,627.655128,530.307692,37.314589,-121.929642
Multi-Family (5+ Unit),1.201936e+07,3.945205,1.666667,20715.584906,37014.125000,1958.125000,653.836478,NaN,37.321021,-121.935989
Other,7.200000e+05,NaN,NaN,NaN,24110.000000,1963.000000,NaN,NaN,37.228370,-121.903792
Single Family Residential,1.585458e+06,3.611657,2.434685,1958.794648,12737.736216,1968.957388,911.690179,126.378887,37.287659,-121.888209
Townhouse,1.090167e+06,2.756898,2.425736,1509.078504,1651.748299,1989.377185,737.025215,341.187286,37.323128,-121.926617
Vacant Land,1.288667e+06,NaN,NaN,NaN,67590.555556,1900.000000,NaN,750.000000,37.273440,-122.014640


In [37]:
redfin_mf5 = redfin.loc[redfin['PROPERTY TYPE'] == 'Multi-Family (5+ Unit)',:]
redfin_mf5.describe()
#More than half of the mf5 type properties are missing beds data and almost none of them
#have baths data.

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
count,1.600000e+02,73.000000,3.000000,159.000000,160.000000,160.000000,159.000000,0.0,160.000000,160.000000
mean,1.201936e+07,3.945205,1.666667,20715.584906,37014.125000,1958.125000,653.836478,NaN,37.321021,-121.935989
std,2.973505e+07,2.391480,1.040833,47882.656353,73313.308087,20.664054,943.753064,NaN,0.095901,0.126733
min,1.835910e+05,1.000000,0.500000,175.000000,3670.000000,1888.000000,53.000000,NaN,37.006381,-122.159193
25%,2.250000e+06,2.000000,1.250000,4168.000000,8619.250000,1956.000000,439.500000,NaN,37.309132,-122.009320
50%,3.894150e+06,3.000000,2.000000,6703.000000,12265.000000,1960.000000,536.000000,NaN,37.340162,-121.940062
75%,8.225000e+06,5.000000,2.250000,13614.500000,29180.000000,1964.000000,644.500000,NaN,37.377336,-121.883939
max,2.555000e+08,11.000000,2.500000,350451.000000,510906.000000,2017.000000,10571.000000,NaN,37.448407,-121.566100


In [38]:
redfin_mf2to4 = redfin.loc[redfin['PROPERTY TYPE'] == 'Multi-Family (2-4 Unit)',:]
redfin_mf2to4.describe()
#70% of the mf2to4 type properties are missing beds data and almost none of them
#have baths data.

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
count,7.820000e+02,562.000000,47.000000,782.000000,7.840000e+02,781.000000,780.000000,13.000000,784.000000,784.000000
mean,1.509862e+06,5.631673,3.292553,2509.993606,9.278583e+03,1957.822023,627.655128,530.307692,37.314589,-121.929642
std,6.478169e+05,1.905408,1.375562,878.980952,3.897761e+04,18.637647,242.567414,911.286488,0.074899,0.103410
min,1.300000e+04,1.000000,1.000000,794.000000,1.523000e+03,1858.000000,8.000000,100.000000,36.993619,-122.155556
25%,1.110500e+06,4.000000,2.000000,1828.000000,6.283000e+03,1956.000000,474.000000,233.000000,37.294032,-121.990850
50%,1.400000e+06,5.000000,3.000000,2366.500000,7.194500e+03,1962.000000,597.500000,300.000000,37.325753,-121.933904
75%,1.800000e+06,7.000000,4.000000,3166.750000,8.583250e+03,1966.000000,737.000000,350.000000,37.353227,-121.876807
max,9.000000e+06,12.000000,7.000000,8630.000000,1.080288e+06,2015.000000,3139.000000,3552.000000,37.448866,-121.511550


In [99]:
#More wrangling on outliers
#Only include property types that are intersting and have enough data
redfin_filtered = redfin[redfin['PROPERTY TYPE'].isin(['Condo/Co-op','Multi-Family (2-4 Unit)',
                                                      'Single Family Residential',
                                                      'Townhouse'])]
redfin_filtered = redfin_filtered.dropna(subset=['SOLD DATE'])
redfin_filtered['QUARTER'] = pd.to_datetime(redfin_filtered['SOLD DATE']).dt.quarter
redfin_filtered = pd.concat([redfin_filtered, pd.get_dummies(redfin_filtered['QUARTER'],
                                                             prefix="Q")], axis=1)  
redfin_filtered

,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE,QUARTER,Q_1,Q_2,Q_3,Q_4
0,July-2-2019,Single Family Residential,788 Orkney Ave,SANTA CLARA,CA,95054,1050000.0,3.0,2.0,1097.0,6242.0,1961.0,957.0,NaN,37.389165,-121.947778,3,0,0,1,0
1,April-19-2018,Condo/Co-op,3901 Lick Mill Blvd #259,SANTA CLARA,CA,95054,882000.0,1.0,1.0,865.0,865.0,2005.0,1020.0,383.0,37.396173,-121.943140,2,0,1,0,0
2,October-20-2017,Condo/Co-op,3901 Lick Mill Blvd #166,SANTA CLARA,CA,95054,680000.0,1.0,1.0,900.0,900.0,2005.0,756.0,390.0,37.396330,-121.943234,4,0,0,0,1
3,October-5-2018,Single Family Residential,956 Baird Ave,SANTA CLARA,CA,95054,1182375.0,4.0,2.0,1466.0,7618.0,1961.0,807.0,NaN,37.387366,-121.950200,4,0,0,0,1
4,May-29-2018,Single Family Residential,3705 Eastwood Cir,SANTA CLARA,CA,95054,1570000.0,4.0,2.0,1666.0,6000.0,1969.0,942.0,NaN,37.392009,-121.940445,2,0,1,0,0
5,February-7-2019,Condo/Co-op,3901 Lick Mill Blvd #254,SANTA CLARA,CA,95054,915000.0,2.0,2.0,1148.0,1148.0,2005.0,797.0,429.0,37.395787,-121.942450,1,1,0,0,0
6,March-4-2019,Condo/Co-op,3901 Lick Mill Blvd #405,SANTA CLARA,CA,95054,805000.0,1.0,1.0,900.0,900.0,2005.0,894.0,400.0,37.396630,-121.942649,1,1,0,0,0
7,November-19-2018,Townhouse,4077 Crandall Cir,SANTA CLARA,CA,95054,1560000.0,3.0,2.5,1894.0,2372.0,2004.0,824.0,422.0,37.397194,-121.945422,4,0,0,0,1
8,August-10-2018,Single Family Residential,3729 De La Cruz Blvd,SANTA CLARA,CA,95054,1210000.0,4.0,2.0,1745.0,6944.0,1967.0,693.0,NaN,37.390199,-121.944778,3,0,0,1,0
9,July-12-2019,Single Family Residential,4949 Avenida De Los Arboles,SANTA CLARA,CA,95054,1230000.0,4.0,2.0,1584.0,6473.0,1976.0,777.0,NaN,37.406260,-121.960575,3,0,0,1,0


In [101]:
#More wrangling on outliers
#Only include property types that are intersting and have enough data
redfin_filtered = redfin[redfin['PROPERTY TYPE'].isin(['Condo/Co-op','Multi-Family (2-4 Unit)',
                                                      'Single Family Residential',
                                                      'Townhouse'])]



#Remove observations with 0 or NAN beds, baths, square feet, lot size, year built
redfin_filtered = redfin_filtered[(redfin_filtered['PRICE'] > 0) & (redfin_filtered['BEDS'] > 0) & 
(redfin_filtered['BATHS'] > 0) & (redfin_filtered['SQUARE FEET'] > 0) & 
(redfin_filtered['LOT SIZE'] > 0) & (redfin_filtered['YEAR BUILT'] > 0)] 

#Replace NaN with 0 for HOA
redfin_filtered['HOA/MONTH'].fillna(0, inplace=True)

redfin_filtered

,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
0,July-2-2019,Single Family Residential,788 Orkney Ave,SANTA CLARA,CA,95054,1050000.0,3.0,2.0,1097.0,6242.0,1961.0,957.0,0.0,37.389165,-121.947778
1,April-19-2018,Condo/Co-op,3901 Lick Mill Blvd #259,SANTA CLARA,CA,95054,882000.0,1.0,1.0,865.0,865.0,2005.0,1020.0,383.0,37.396173,-121.943140
2,October-20-2017,Condo/Co-op,3901 Lick Mill Blvd #166,SANTA CLARA,CA,95054,680000.0,1.0,1.0,900.0,900.0,2005.0,756.0,390.0,37.396330,-121.943234
3,October-5-2018,Single Family Residential,956 Baird Ave,SANTA CLARA,CA,95054,1182375.0,4.0,2.0,1466.0,7618.0,1961.0,807.0,0.0,37.387366,-121.950200
4,May-29-2018,Single Family Residential,3705 Eastwood Cir,SANTA CLARA,CA,95054,1570000.0,4.0,2.0,1666.0,6000.0,1969.0,942.0,0.0,37.392009,-121.940445
5,February-7-2019,Condo/Co-op,3901 Lick Mill Blvd #254,SANTA CLARA,CA,95054,915000.0,2.0,2.0,1148.0,1148.0,2005.0,797.0,429.0,37.395787,-121.942450
6,March-4-2019,Condo/Co-op,3901 Lick Mill Blvd #405,SANTA CLARA,CA,95054,805000.0,1.0,1.0,900.0,900.0,2005.0,894.0,400.0,37.396630,-121.942649
7,November-19-2018,Townhouse,4077 Crandall Cir,SANTA CLARA,CA,95054,1560000.0,3.0,2.5,1894.0,2372.0,2004.0,824.0,422.0,37.397194,-121.945422
8,August-10-2018,Single Family Residential,3729 De La Cruz Blvd,SANTA CLARA,CA,95054,1210000.0,4.0,2.0,1745.0,6944.0,1967.0,693.0,0.0,37.390199,-121.944778
9,July-12-2019,Single Family Residential,4949 Avenida De Los Arboles,SANTA CLARA,CA,95054,1230000.0,4.0,2.0,1584.0,6473.0,1976.0,777.0,0.0,37.406260,-121.960575


In [64]:
redfin_filtered.describe()

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
count,2.736600e+04,27366.000000,27366.000000,27366.000000,2.736600e+04,27366.000000,2.736000e+04,27366.000000,27366.000000,27366.000000
mean,1.401080e+06,3.292187,2.344734,1791.790105,9.882674e+03,1973.970255,8.847797e+02,109.032814,37.296461,-121.895733
std,9.452679e+05,1.001874,0.843225,852.284805,4.160619e+04,22.965801,1.106975e+04,190.357931,0.094026,0.128900
min,3.895000e+03,1.000000,1.000000,1.000000,1.000000e+00,1858.000000,3.000000e+00,0.000000,36.964850,-122.180281
25%,8.599132e+05,3.000000,2.000000,1240.000000,2.863000e+03,1959.000000,5.687500e+02,0.000000,37.254675,-121.981586
50%,1.165000e+06,3.000000,2.000000,1592.000000,6.077000e+03,1972.000000,7.290000e+02,0.000000,37.307560,-121.898671
75%,1.600000e+06,4.000000,3.000000,2097.750000,7.942750e+03,1988.000000,9.570000e+02,215.000000,37.360641,-121.830522
max,2.349500e+07,10.000000,10.000000,12694.000000,2.652368e+06,2019.000000,1.730000e+06,6000.000000,37.465350,-121.419033


In [65]:
#Check the 1st, 10th, 90th, and 99th percentiles see if any outliers
redfin_filtered.quantile([.01, .1, .9, .99])

,PRICE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
0.01,430000.0,1.0,1.0,704.00,458.0,1908.0,305.59,0.0,37.000478,-122.147900
0.10,676000.0,2.0,1.0,1010.00,1077.0,1950.0,454.00,0.0,37.203725,-122.058098
0.90,2400000.0,4.0,3.5,2769.00,11826.5,2006.0,1222.00,392.0,37.399419,-121.748743
0.99,4900000.0,6.0,5.0,5048.25,97525.7,2018.0,1911.64,704.0,37.445958,-121.575138


In [30]:
#Read Airbnb data
airbnb = pd.read_csv(data_path + "Airbnb/Santa Clara/listings.csv")

pd.set_option('display.max_columns', None)
airbnb.head(2)
#Columns removed:listing_url, scrape_id, name, summary, space, description, experiences_offered(all none), neighborhood_overview,
#                notes, transit, access, interaction, house_rules, thumbnail_url, medium_url, picture_url, xl_picture_url,
#                host_id, host_url, host_name, host_location, host_about, host_acceptance_rate, host_thumbnail_url, 
#                host_picture_url, host_neighborhood, host_total_listings_count, host_verifications, neighbourhood_group_cleansed,
#                market, smart_location, country_code, country, is_location_exact, 'has_availability', 'calendar_last_scraped',
#                requires_license, license, jurisdiction_names, is_business_travel_ready

#for some of the texts, might be worth extracting some new features? like no-smoking
#Link the calendar table for seaonal adjusted prices?
#min-min (calendar?)
#availability could be because the host doens't want to list it

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,4952,https://www.airbnb.com/rooms/4952,20190709175204,2019-07-09,Butterfly Inn - Graceful Living !,Lovely garden setting in a serene and art-fill...,Very comfortable Queen bed and small desk in b...,Lovely garden setting in a serene and art-fill...,none,"Located in Professorville, Palo Alto, one of t...","Housekeeping every Monday, leave your door ope...",Walking distance to Stanford University (30 mi...,"Kitchen, laundry, family/TV room, garden, free...","Welcome and Orientation. Shared kitchen, garde...",No smoking in the house or garden Shared bathr...,NaN,NaN,https://a0.muscache.com/im/pictures/529448/3d7...,NaN,7054,https://www.airbnb.com/users/show/7054,Mei Lin,2009-01-24,"Palo Alto, California, United States","I have lived and worked in Asia, Europe, Austr...",within an hour,100%,NaN,t,https://a0.muscache.com/im/users/7054/profile_...,https://a0.muscache.com/im/users/7054/profile_...,Palo Alto,5.0,5.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Palo Alto, CA, United States",Palo Alto,Palo Alto,NaN,Palo Alto,CA,94301,"South Bay, CA","Palo Alto, CA",US,United States,37.43972,-122.15672,t,Villa,Private room,2,2.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free str...",NaN,$90.00,$440.00,"$1,428.00",$0.00,$35.00,1,$45.00,2,365,2,2,365,365,2.0,365.0,3 days ago,t,0,0,0,4,2019-07-09,75,2,2009-08-02,2019-01-12,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,"{""PALO ALTO""}",f,f,moderate,f,f,5,0,5,0,0.62
1,11464,https://www.airbnb.com/rooms/11464,20190709175204,2019-07-09,Deluxe Private Studio-custom int.,Custom built Studio with exquisite design. Per...,Description A favorite for international corpo...,Custom built Studio with exquisite design. Per...,none,10 min. to Lucky's and Starbucks at El Camino ...,Pet Policy: - Well Behaved pet up to 25 lb. of...,Public transportation at Homestead and Pomeroy...,Complimentary Wifi-internet + Basic Cable,Available any time for support and problem res...,Can house a couple- bed type is a full size be...,NaN,NaN,https://a0.muscache.com/im/pictures/121321/2c5...,NaN,42458,https://www.airbnb.com/users/show/42458,Emey,2009-09-30,"Mountain View, California, United States",I pride myself on attention to details and cus...,within a few hours,100%,NaN,f,https://a0.musc

In [43]:
#Keep the relevant columns
airbnb = airbnb[['id', 'last_scraped', 'host_since', 'host_response_time', 'host_response_rate', 'host_is_superhost',
                'host_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood',
                'neighbourhood_cleansed', 'city', 'state', 'zipcode', 'latitude', 'longitude', 'property_type',
                'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
                'price', 'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included',
                'extra_people', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
                'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
                'calendar_updated', 'availability_30',
                'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
                'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value',
                'instant_bookable', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification',
                'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'reviews_per_month',
                'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']]

airbnb.head()

,id,last_scraped,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,reviews_per_month,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,4952,2019-07-09,2009-01-24,within an hour,100%,t,5.0,t,f,"Palo Alto, CA, United States",Palo Alto,Palo Alto,Palo Alto,CA,94301,37.43972,-122.15672,Villa,Private room,2,2.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Free str...",NaN,$90.00,$440.00,"$1,428.00",$0.00,$35.00,1,$45.00,2,365,2,2,365,365,2.0,365.0,3 days ago,0,0,0,4,75,2,2009-08-02,2019-01-12,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,moderate,f,f,5,0,0.62,5,0
1,11464,2019-07-09,2009-09-30,within a few hours,100%,f,14.0,t,f,"Santa Clara, CA, United States",Santa Clara,Santa Clara,Santa Clara,CA,95051,37.34342,-121.98829,Apartment,Entire home/apt,1,1.0,NaN,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",325.0,$85.00,NaN,"$2,100.00",$700.00,$250.00,1,$10.00,14,365,14,14,365,365,14.0,365.0,a week ago,0,6,36,311,5,2,2014-06-15,2018-08-26,84.0,10.0,10.0,10.0,9.0,9.0,9.0,f,moderate,f,t,13,12,0.08,1,0
2,21373,2019-07-09,2009-01-24,within an hour,100%,t,5.0,t,f,"Palo Alto, CA, United States",Palo Alto,Palo Alto,Palo Alto,CA,94301,37.44001,-122.15667,Villa,Private room,1,2.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Paid par...",NaN,$99.00,$600.00,"$2,000.00",$0.00,$35.00,1,$100.00,2,730,2,2,730,730,2.0,730.0,3 days ago,1,14,22,275,231,58,2010-05-28,2019-06-28,97.0,10.0,10.0,10.0,10.0,10.0,10.0,f,moderate,f,f,5,0,2.08,5,0
3,23732,2019-07-09,2008-08-15,NaN,NaN,f,1.0,t,t,"Mountain View, CA, United States",Mountain View,Mountain View,Mountain View,CA,94043,37.39203,-122.06675,House,Private room,2,1.0,1.0,1.0,Futon,"{TV,Internet,Wifi,Kitchen,""Washer / Dryer"",""In...",NaN,$299.00,NaN,NaN,$500.00,$75.00,1,$25.00,1,365,1,1,365,365,1.0,365.0,57 months ago,30,60,90,365,3,0,2010-05-20,2011-07-07,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,moderate,f,f,1,0,0.03,1,0
4,29790,2019-07-09,2010-05-19,within an hour,100%,f,1.0,t,t,"Mountain View, CA, United States",Mountain View,Mountain View,Mountain View,CA,94041,37.38955,-122.07905,House,Private room,2,1.0,1.0,1.0,Futon,"{TV,Wifi,Heating,""Family/kid friendly"",Washer,...",NaN,$99.00,NaN,NaN,$200.00,$40.00,2,$0.00,2,16,2,2,16,16,2.0,16.0,4 weeks ago,0,0,6,6,65,6,2013-08-02,2019-05-11,92.0,9.0,9.0,10.0,10.0,10.0,9.0,f,flexible,f,f,1,0,0.90,1,0


In [44]:
airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7196 entries, 0 to 7195
Data columns (total 66 columns):
id                                              7196 non-null int64
last_scraped                                    7196 non-null object
host_since                                      7194 non-null object
host_response_time                              6463 non-null object
host_response_rate                              6463 non-null object
host_is_superhost                               7194 non-null object
host_listings_count                             7194 non-null float64
host_has_profile_pic                            7194 non-null object
host_identity_verified                          7194 non-null object
street                                          7196 non-null object
neighbourhood                                   6718 non-null object
neighbourhood_cleansed                          7196 non-null object
city                                            7187 non-null obj

In [45]:
airbnb.describe()

,id,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,reviews_per_month,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
count,7.196000e+03,7194.000000,7196.000000,7196.000000,7196.000000,7195.000000,7195.000000,7190.000000,29.000000,7196.000000,7196.000000,7.196000e+03,7196.000000,7196.000000,7.196000e+03,7.196000e+03,7196.000000,7.196000e+03,7196.000000,7196.000000,7196.000000,7196.000000,7196.000000,7196.000000,5858.000000,5857.000000,5857.000000,5855.000000,5856.000000,5856.000000,5855.000000,7196.000000,7196.000000,5906.000000,7196.000000,7196.000000
mean,2.192405e+07,85.044760,37.354823,-121.973816,3.215258,1.410215,1.456150,1.907789,813.310345,1.708310,7.426765,3.283176e+05,6.937187,8.246943,3.283146e+05,3.283147e+05,7.712535,3.283147e+05,7.606031,22.979572,42.776681,178.805309,27.604086,12.420650,95.270058,9.721188,9.578965,9.812810,9.806694,9.770833,9.537660,26.061145,21.961228,1.743493,2.247916,1.852001
std,1.027908e+07,282.907513,0.064165,0.108431,2.442318,0.719481,1.031403,1.555120,621.796769,1.611665,20.086237,2.537012e+07,19.722634,20.494085,2.537012e+07,2.537012e+07,20.073603,2.537012e+07,9.822073,19.888564,29.761746,132.772453,45.757737,19.070046,7.517582,0.759019,0.827892,0.661659,0.656723,0.605477,0.798385,73.800338,74.066452,1.839373,4.169210,8.929057
min,4.952000e+03,0.000000,36.965600,-122.197880,1.000000,0.000000,0.000000,0.000000,100.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.010000,0.000000,0.000000
25%,1.362560e+07,1.000000,37.321625,-122.063413,2.000000,1.000000,1.000000,1.000000,225.000000,1.000000,1.000000,6.000000e+01,1.000000,1.000000,6.000000e+01,6.000000e+01,1.000000,6.000000e+01,0.000000,3.000000,14.000000,50.000000,1.000000,1.000000,94.000000,10.000000,9.000000,10.000000,10.000000,10.000000,9.000000,1.000000,0.000000,0.480000,0.000000,0.000000
50%,2.360020e+07,3.000000,37.363435,-121.968270,2.000000,1.000000,1.000000,1.000000,800.000000,1.000000,2.000000,1.125000e+03,2.000000,2.000000,1.125000e+03,1.125000e+03,2.000000,1.125000e+03,3.000000,21.000000,45.000000,163.500000,10.000000,5.000000,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,1.000000,1.080000,1.000000,0.000000
75%,3.072629e+07,13.000000,37.399932,-121.892507,4.000000,2.000000,2.000000,2.000000,1300.000000,2.000000,4.000000,1.125000e+03,4.000000,5.000000,1.125000e+03,1.125000e+03,4.800000,1.125000e+03,13.000000,38.000000,67.000000,319.000000,33.000000,16.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,2.370000,2.000000,0.000000
max,3.653630e+07,1295.000000,37.462950,-121.380120,16.000000,8.000000,10.000000,26.000000,2500.000000,16.000000,365.000000,2.147484e+09,365.000000,365.000000,2.147484e+09,2.147484e+09,365.000000,2.147484e+09,30.000000,60.000000,90.000000,365.000000,446.000000,174.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,344.000000,344.000000,15.430000,31.000000,63.000000
